# Task 2
## 1. Env set up
1. ... (already done before)
2. pip install scikit-learn(including sklearn, scipy) torch
---
## 2. Learning/Working route
1. figure out task-involved knowledge range, in this case package calling
2. watch some short videos about these package to quickly get knowing what they do, have, and their advantages
3. check their usage in the official documents
4. learn new concepts like OneHot encoding, ask AI about its common realizations, and learn related methods
5. ask AI about common ways of processing numerical data and do it under guidance
6. when visualizing, using AI to help adjust the parameters
---
## 3. Work Sequentially
#### a. Before start
- numpy can read only numerical data, still use pandas to read
- preprocess before further action:
    1. filling missing data(with mean)
    2. encode categorical features and 'international' col(using LabelEncoder)
    3. prepare target var

In [39]:
import numpy as np
import scipy as sp
import sklearn as sk
import pandas as pd

# read data
train_data = pd.read_csv('../MBAAdmission/train.csv')
test_data = pd.read_csv('../MBAAdmission/test.csv')

# preprocess data
def preprocess(data):
    # handle missing values
    num_cols = data.select_dtypes(include=[np.number]).columns
    data[num_cols] = data[num_cols].fillna(data[num_cols].mean())
    for col in data.select_dtypes(include=[object]).columns:
        if col != 'admission':
            data[col]=data[col].fillna(data[col].mode()[0])
    return

preprocess(train_data)
preprocess(test_data)

categorical_cols = train_data.select_dtypes(include=[object]).columns
numeric_cols = train_data.select_dtypes(include=[np.number]).columns

# prepare features and labels

# encode categorical variables
X_train = train_data.drop(columns=['application_id', 'admission'])
X_test = test_data.drop(columns=['application_id', 'admission'])

# simple label encoding for categorical variables
for col in categorical_cols:
    if col in X_train.columns:
        le = sk.preprocessing.LabelEncoder()
        X_train[col] = le.fit_transform(X_train[col].astype(str))
        # handle unseen labels in test set
        if col in X_test.columns:
            X_test[col] = X_test[col].astype(str)
            X_test[col] = X_test[col].map(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
            
# encode 'international' column
X_train['international'] = X_train['international'].astype(int)
X_test['international'] = X_test['international'].astype(int)

# target variable
y_encoder = sk.preprocessing.LabelEncoder()
y_train = y_encoder.fit_transform(train_data['admission'])

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Classes: {y_encoder.classes_}")

Training set shape: (6095, 8)
Test set shape: (99, 8)
Classes: ['Admit' 'Reject' 'Waitlist']


C:\Users\zxy84\AppData\Local\Temp\ipykernel_30948\738569145.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col]=data[col].fillna(data[col].mode()[0])


#### b. Subtask 1
- call sk..LinearRegression, fit, predict and assess.
- call LogisticReg, got warned that 1000 iter woundn't lead to convergence,
try 3000, the same; 4000, converged.
- output accuracy on both sets.


In [40]:
# Subtask 1: Linear and Logistic Regression
# a. Linear Regression
print("\n--- Linear Regression ---")
linear_reg = sk.linear_model.LinearRegression()
linear_reg.fit(X_train, y_train)

# make predictions
y_train_pred_linear = linear_reg.predict(X_train)
y_test_pred_linear = linear_reg.predict(X_test)

# convert to int for classification
y_train_pred_linear_class = np.round(np.clip(y_train_pred_linear, 0, len(y_encoder.classes_)-1)).astype(int)
y_test_pred_linear_class = np.round(np.clip(y_test_pred_linear, 0, len(y_encoder.classes_)-1)).astype(int)

# Calculate accuracy
train_acc_linear = sk.metrics.accuracy_score(y_train, y_train_pred_linear_class)
test_acc_linear = sk.metrics.accuracy_score(y_encoder.transform(test_data['admission']), y_test_pred_linear_class)

print(f"   Training Accuracy: {train_acc_linear:.4f}")
print(f"   Test Accuracy: {test_acc_linear:.4f}")
print(f"   Linear regression coefficients shape: {linear_reg.coef_.shape}")


# b. Logistic Regression
print("\n--- Logistic Regression ---")
logistic_reg = sk.linear_model.LogisticRegression(max_iter=4000, random_state=42)
logistic_reg.fit(X_train, y_train)

# make predictions
y_train_pred_logistic = logistic_reg.predict(X_train)
y_test_pred_logistic = logistic_reg.predict(X_test)

# Calculate accuracy
train_acc_logistic = sk.metrics.accuracy_score(y_train, y_train_pred_logistic)
test_acc_logistic = sk.metrics.accuracy_score(y_encoder.transform(test_data['admission']), y_test_pred_logistic)

print(f"   Training Accuracy: {train_acc_logistic:.4f}")
print(f"   Test Accuracy: {test_acc_logistic:.4f}")
print(f"   Logistic regression coefficients shape: {logistic_reg.coef_.shape}")

print("\nRegression Done.")


--- Linear Regression ---
   Training Accuracy: 0.8466
   Test Accuracy: 0.3333
   Linear regression coefficients shape: (8,)

--- Logistic Regression ---
   Training Accuracy: 0.8409
   Test Accuracy: 0.3838
   Logistic regression coefficients shape: (3, 8)

Regression Done.


#### c. Subtask 2
- normalize for better training
- build model, thanks AI for helping fill in the parameters
- Glad to see 'adam' again btw.

In [41]:
# Subtask 2: sklearn MLP Classifier
print("\n--- MLP Classifier ---")

# normalize features
scaler = sk.preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Scaled Training set shape: {X_train_scaled.shape}")
print(f"Scaled Test set shape: {X_test_scaled.shape}")

# build and train MLP using provided parameters
input_dim = X_train_scaled.shape[1]
output_dim = len(y_encoder.classes_)

print(f"Network Architecture: [{input_dim}, 128] -> [128, 256] -> [256, {output_dim}]")

# build model
mlp = sk.neural_network.MLPClassifier(
    hidden_layer_sizes=(128, 256),  # Two hidden layers: 128 and 256 neurons
    activation='relu',              # ReLU activation
    learning_rate_init=0.0001,      # lr = 0.0001
    max_iter=50,                    # epoch = 50
    batch_size=64,                  # batch = 64
    random_state=42,
    solver='adam',                  # Adam optimizer (sklearn default)
    early_stopping=False,           # Don't stop early to complete all epochs
    verbose=True                    # Show training progress
)

# train model
print("Training MLP...")
mlp.fit(X_train_scaled, y_train)
print("MLP Training Complete.")

# make predictions
y_train_pred_mlp = mlp.predict(X_train_scaled)
y_test_pred_mlp = mlp.predict(X_test_scaled)

# Calculate accuracy
train_acc_mlp = sk.metrics.accuracy_score(y_train, y_train_pred_mlp)
test_acc_mlp = sk.metrics.accuracy_score(y_encoder.transform(test_data['admission']), y_test_pred_mlp)
print(f"   Training Accuracy: {train_acc_mlp:.4f}")
print(f"   Test Accuracy: {test_acc_mlp:.4f}")
total_params = sum(coef.size for coef in mlp.coefs_) + sum(bias.size for bias in mlp.intercepts_)
print(f"   Total parameters: {total_params}")
print("MLP Done.")


--- MLP Classifier ---
Scaled Training set shape: (6095, 8)
Scaled Test set shape: (99, 8)
Network Architecture: [8, 128] -> [128, 256] -> [256, 3]
Training MLP...
Iteration 1, loss = 0.67510977
Iteration 2, loss = 0.43760726
Iteration 3, loss = 0.40227626
Iteration 4, loss = 0.39255596
Iteration 5, loss = 0.38850201
Iteration 6, loss = 0.38602716
Iteration 7, loss = 0.38351205
Iteration 8, loss = 0.38231942
Iteration 9, loss = 0.38056649
Iteration 10, loss = 0.37944881
Iteration 11, loss = 0.37815923
Iteration 12, loss = 0.37692867
Iteration 13, loss = 0.37576888
Iteration 14, loss = 0.37473733
Iteration 15, loss = 0.37371770
Iteration 16, loss = 0.37291974
Iteration 17, loss = 0.37199046
Iteration 18, loss = 0.37133703
Iteration 19, loss = 0.37025144
Iteration 20, loss = 0.36922436
Iteration 21, loss = 0.36881741
Iteration 22, loss = 0.36800906
Iteration 23, loss = 0.36734866
Iteration 24, loss = 0.36671220
Iteration 25, loss = 0.36567163
Iteration 26, loss = 0.36491075
Iteration 27

c:\scoop\apps\python\3.13.7\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


#### d. Subtask 3
- install torch
- search and see what 'super' class is

In [ ]:
# Subtask 3: Torch MLP Classifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# define MLP model
class MLP_PyTorch(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP_PyTorch, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256), 
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.network(x)
